# Test rotation similarity and finidng unique angle

Since it is possible to get the same 4D matrix from a two different sets of 6D vectors, in this notebook we want to test and explore different ways of getting a unique 6D vector values from a 4D rotation matrix and define a standard that will be used in the future code. 

Simple example in a lower dimension: if we are in 3D space, and we rotate an object around x-axis by 90 degrees and then around y-axis by 90 degrees, it would have the same position if instead we rotated an object around z-axis by 90 degrees. In our problem, we are working in a 4D space, but similar rules should apply there as well.

### Content

- [1. Test rotation similarity](#1.-Test-rotation-similarity)
    - [1.1. Test with Alignment methods](#1.1.-Test-with-Alignment-methods)
- [2. Finding unique angle](#2.-Finding-unique-angle)
- [3. Solving 16 equations (from a 4D rotation matrix)](#3.-Solving-16-equations-(from-a-4D-rotation-matrix))

---

## 1. Test rotation similarity

In [1]:
import numpy as np

In the following image we are trying to empirically find what is the rule that connects the sets of 6D vectors that give the same 4D rotation matrix. After finding the posible rule, we are running several tests. The method that is implemented is the first method we developed to compare if two 6D vectors give the same 4D rotation, called `rotations_equal(R1, R2)`.

![](https://i.imgur.com/jabkxNt.jpg)

In [2]:
R1 = [7.46021406, 2.58905189, 5.9320024,  3.06165002, 2.56025736, 3.82164765]
R2 = [1.17798222, 5.7311303,  3.49258011, 6.36339684, 5.70202671, 3.82134471]
R3 = [-1.96368164,  3.69437634,  3.492642 ,   3.22182489, -0.58032233,  3.82173555]

In [3]:
R4 = [0, 0, 0, 0, 0, 0]
R5 = [0, 2*np.pi, np.pi, 0, np.pi, np.pi]
R6 = [-np.pi, -np.pi, -np.pi, 0, 0, 0]

In [216]:
def rotations_equal(R1, R2):
    """Compare two 6D rotation vectors
    
    Parameters
    ----------
    R1: 6d np.ndarray
        First 6D vector
    R2: 6d np.ndarray
        Second 6D vector
    
    Returns
    -------
    : bool
        True if two sets of 6d vecotrs have the same 4D rotation, False otherwise.
    """
    tol = 1e-2
    
    R1 = list(map(lambda x: x%(2*np.pi), R1))
    R2 = list(map(lambda x: x%(2*np.pi), R2))
    
    r = lambda i: R1[i]+R2[i]-2*(np.allclose(np.abs(R1[i]-R2[i])%np.pi, 0.0, atol=tol) or np.allclose(np.abs(R1[i]-R2[i])%np.pi, np.pi, atol=tol))*R2[i] 

    rd = lambda i, j: (r(i)+r(j))%(2*np.pi) if not np.allclose((r(i)+r(j))%(2*np.pi), 2*np.pi, atol=tol) else \
                      (2*np.pi)%(r(i)+r(j))

    return np.allclose((rd(0, 5) + rd(1, 4) + rd(2, 3))%(3*np.round(np.pi,2)),     0.0, atol=tol) or \
           np.allclose((rd(0, 5) + rd(1, 4) + rd(2, 3))%(3*np.round(np.pi,2)), 3*np.pi, atol=tol) 

In [218]:
rotations_equal(R1, R2), rotations_equal(R2, R3), rotations_equal(R1, R3) #== (True, True, True)

(True, True, True)

In [219]:
rotations_equal(R4, R5), rotations_equal(R4, R6), rotations_equal(R5, R6) #== (True, True, True)

(True, True, True)

In [220]:
rotations_equal(R1, R5), rotations_equal(R1, R6), rotations_equal(R2, R4) #== (False, False, False)

(False, False, False)

In [221]:
rotations_equal(R1, R4), rotations_equal(R2, R4), rotations_equal(R2, R5) #== (False, False, False)

(False, False, False)

In [222]:
Rd1 = [ 2.55273739,  2.54278743, -0.31850297, -0.41680098,  3.52962884, -0.36112232]
Rd2 = [ 2.92116975,  2.550321  , -0.28551178,  2.68878358, -0.40685398, 2.77110005]
Rd3 = [ 2.80438687,  0.66014735,  0.05556158,  2.9211356 ,  3.7135224 , 0.21344568]

In [223]:
rotations_equal(Rd1, Rd2), rotations_equal(Rd2, Rd3), rotations_equal(Rd1, Rd3) #== (True, True, True)

(False, False, False)

### 1.1. Test with Alignment methods

In [1]:
import sys
sys.path.append("..") 
from oml.alignment import update_quaternion
from oml.angles import quaternion2euler, euler2quaternion

In [10]:
ang = [1.0, 2.0, 3.0]

rotation = [np.array([0, 2*np.pi, np.pi, 0, np.pi, np.pi], dtype=np.float64)]

qp = euler2quaternion(np.array([ang], dtype=np.float64))

qpa = update_quaternion(m=[1.0,1.0,1.0,1.0], 
                        a_R=rotation, 
                        q_predicted=qp)

angles_predicted = quaternion2euler(qpa).numpy()

angles_predicted

array([[1., 2., 3.]])

In [11]:
ang = [1.0, 2.0, 3.0]

rotation = [np.array([np.pi, np.pi, np.pi, np.pi, np.pi, np.pi], dtype=np.float64)]

qp = euler2quaternion(np.array([ang], dtype=np.float64))

qpa = update_quaternion(m=[1.0,1.0,1.0,1.0], 
                        a_R=rotation, 
                        q_predicted=qp)

angles_predicted = quaternion2euler(qpa).numpy()

angles_predicted

array([[1., 2., 3.]])

In [12]:
ang = [1.0, 2.0, 3.0]

rotation = [np.array(R1, dtype=np.float64)]

qp = euler2quaternion(np.array([ang], dtype=np.float64))

qpa = update_quaternion(m=[1.0,1.0,1.0,1.0], 
                        a_R=rotation, 
                        q_predicted=qp)

angles_predicted = quaternion2euler(qpa).numpy()

angles_predicted

array([[ 2.29822589,  2.94827061, -2.48492839]])

In [13]:
ang = [1.0, 2.0, 3.0]

rotation = [np.array(R2, dtype=np.float64)]

qp = euler2quaternion(np.array([ang], dtype=np.float64))

qpa = update_quaternion(m=[1.0,1.0,1.0,1.0], 
                        a_R=rotation, 
                        q_predicted=qp)

angles_predicted = quaternion2euler(qpa).numpy()

angles_predicted

array([[ 2.29429832,  2.94822796, -2.49014507]])

In [14]:
ang = [1.0, 2.0, 3.0]

rotation = [np.array(R3, dtype=np.float64)]

qp = euler2quaternion(np.array([ang], dtype=np.float64))

qpa = update_quaternion(m=[1.0,1.0,1.0,1.0], 
                        a_R=rotation, 
                        q_predicted=qp)

angles_predicted = quaternion2euler(qpa).numpy()

angles_predicted

array([[ 2.30144463,  2.94862414, -2.48245127]])

---

## 2. Finding unique angle

In this part of the notebook we are finding 6d vecor representative. Since many combinations of 6d vectors give the same 4D rotation, here we define a way to single out one of them to represent their group. This is done in the method called `create_unique_angle(point)`. It constructs the equivalent 6d vector with all positive components such that their sum is minimum.

In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
def euler6tomarix4d(a_R):
    """ Convert 6D vector containing angles to 4D rotation matrix

    a_R: tf.tensor/np.ndarray
        Vector of shape (6,)

    Returns
    -------
    R: tf.tensor/np.ndarray
        4x4 Rotation matrix corresponding to these 6 angles of rotations
    """
    xy, xz, xw, yz, yw, zw = a_R

    cxy = np.cos(xy)
    cxz = np.cos(xz)
    cxw = np.cos(xw)
    cyz = np.cos(yz)
    cyw = np.cos(yw)
    czw = np.cos(zw)

    sxy = np.sin(xy)
    sxz = np.sin(xz)
    sxw = np.sin(xw)
    syz = np.sin(yz)
    syw = np.sin(yw)
    szw = np.sin(zw)

    # Note: wasn't able to create it as simple as np.ndarrays...
    Rxy = np.array([[  cxy,  -sxy, 0.0, 0.0],
              [  sxy,   cxy, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

    Rxz = np.array([[  cxz, 0.0,  -sxz, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [  sxz, 0.0,   cxz, 0.0],
               [0.0, 0.0, 0.0, 1.0]])

    Rxw = np.array([[  cxw, 0.0, 0.0,  -sxw],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0, 1.0, 0.0],
               [  sxw, 0.0, 0.0,  cxw]])

    Ryz = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0,   cyz,  -syz, 0.0],
               [0.0,   syz,   cyz, 0.0],
               [0.0, 0.0, 0.0, 1.0]])

    Ryw = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0,   cyw, 0.0,  -syw],
               [0.0, 0.0, 1.0, 0.0],
               [0.0,   syw, 0.0,  cyw]])

    Rzw = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0,   czw,  -szw],
               [0.0, 0.0,   szw,  czw]])

    R = Rxy @ Rxz @ Rxw @ Ryz @ Ryw @ Rzw

    return R

In [3]:
df = pd.read_csv("../results/alignment/loss.csv")
df['seed'] = df.seed.astype(int)
df['good_conv'] = (df['final_loss'] < 0.1)

In [4]:
n_runs = 100
trajectories = {}
critical_points = np.zeros([n_runs, 6])
for i in range(n_runs):
    trajectories[i] = np.load(f'../results/alignment/trajectories/{i}.npy')
    critical_points[i, :] = trajectories[i][-1, :]

In [5]:
def rotations_equal(R1, R2):
    tol = 1e-8
    
    R1 = list(map(lambda x: x%(2*np.pi), R1))
    R2 = list(map(lambda x: x%(2*np.pi), R2))
    
    r = lambda i: R1[i]+R2[i]-2*(np.allclose(np.abs(R1[i]-R2[i])%np.pi, 0.0, atol=tol) or np.allclose(np.abs(R1[i]-R2[i])%np.pi, np.pi, atol=tol))*R2[i] 

    rd = lambda i, j: (r(i)+r(j))%(2*np.pi) if not np.allclose((r(i)+r(j))%(2*np.pi), 2*np.pi, atol=tol) else \
                      (2*np.pi)%(r(i)+r(j))

    return np.allclose((rd(0, 5) + rd(1, 4) + rd(2, 3))%(3*np.round(np.pi,2)),     0.0, atol=tol) or \
           np.allclose((rd(0, 5) + rd(1, 4) + rd(2, 3))%(3*np.round(np.pi,2)), 3*np.pi, atol=tol)

In [6]:
def rotations_equal(R1, R2, return_m=False):
    M1 = euler6tomarix4d(R1)
    M2 = euler6tomarix4d(R2)
    if return_m:
        return M1, M2
    cond = (np.allclose(M1, M2) | np.allclose(M1, -M2))
    return cond

In [7]:
def create_unique_angle(point):
    """Constructs the equivalent 6d vector with all positive components
    such that their sum is minimum"""
    point = [x % (2 * np.pi) for x in point]
    return point

In [8]:
a = [4.268, 5.68,  3.375, 3.562, 0.445, 6.004]
print(f"{np.around(a,3)}")
print(sum(a))

[4.268 5.68  3.375 3.562 0.445 6.004]
23.334000000000003


In [9]:
b = create_unique_angle(a)
print(f"{np.around(b,3)}")
print(sum(b))

[4.268 5.68  3.375 3.562 0.445 6.004]
23.334000000000003


In [10]:
c = b.copy()
c[0] = 0
c[5] += b[0]

In [11]:
print(f"{np.around(a,3)}")
print(f"{np.around(b,3)}")
print(f"{np.around(c,3)}")
print(rotations_equal(a,b))
print(rotations_equal(a,c))
print(rotations_equal(b,c))

[4.268 5.68  3.375 3.562 0.445 6.004]
[4.268 5.68  3.375 3.562 0.445 6.004]
[ 0.     5.68   3.375  3.562  0.445 10.272]
True
False
False


In [13]:
df

,seed,final_loss,good_conv
0,0,1.804267,False
1,1,0.002206,True
2,2,0.002551,True
3,3,1.813860,False
4,4,1.805387,False
...,...,...,...
95,95,0.002384,True
96,96,0.002349,True
97,97,1.808804,False
98,98,0.002554,True


In [14]:
a = critical_points[1, :]
a

array([7.46021406, 2.58905189, 5.9320024 , 3.06165002, 2.56025736,
       3.82164765])

In [15]:
b = critical_points[2, :]
b

array([1.17798222, 5.7311303 , 3.49258011, 6.36339684, 5.70202671,
       3.82134471])

In [16]:
c = create_unique_angle(a)
np.array(c)

array([1.17702875, 2.58905189, 5.9320024 , 3.06165002, 2.56025736,
       3.82164765])

In [17]:
d = create_unique_angle(b)
np.array(d)

array([1.17798222, 5.7311303 , 3.49258011, 0.08021153, 5.70202671,
       3.82134471])

In [19]:
funs = [idd, opp, add_pi, opp_pi]
list_comb = []
it = [0, 0, 0, 0, 0, 0, 0]
while it[0] == 0:
    e = np.empty_like(a)
    for i in range(6):
        e[i] = funs[it[i + 1]](a[i])
    if rotations_equal(a, e):
        list_comb.append(it[1:])
    it[-1] += 1
    for i in range(6, -1, -1):
        if it[i] == 4:
            it[i] = 0
            it[i-1] += 1
list_comb

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 3, 2],
 [0, 0, 2, 0, 3, 2],
 [0, 0, 2, 2, 0, 0],
 [0, 2, 1, 1, 2, 0],
 [0, 2, 1, 3, 1, 2],
 [0, 2, 3, 1, 1, 2],
 [0, 2, 3, 3, 2, 0],
 [2, 1, 1, 1, 1, 2],
 [2, 1, 1, 3, 2, 0],
 [2, 1, 3, 1, 2, 0],
 [2, 1, 3, 3, 1, 2],
 [2, 3, 0, 0, 3, 2],
 [2, 3, 0, 2, 0, 0],
 [2, 3, 2, 0, 0, 0],
 [2, 3, 2, 2, 3, 2]]

In [20]:
def create_unique_angle(point):
    """Constructs the equivalent 6d vector with all positive components
    such that their sum is minimum"""
    LIST_FUN = [idd, opp, add_pi, opp_pi]
    LIST_EQUIV = [
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 3, 2],
        [0, 0, 2, 0, 3, 2],
        [0, 0, 2, 2, 0, 0],
        [0, 2, 1, 1, 2, 0],
        [0, 2, 1, 3, 1, 2],
        [0, 2, 3, 1, 1, 2],
        [0, 2, 3, 3, 2, 0],
        [2, 1, 1, 1, 1, 2],
        [2, 1, 1, 3, 2, 0],
        [2, 1, 3, 1, 2, 0],
        [2, 1, 3, 3, 1, 2],
        [2, 3, 0, 0, 3, 2],
        [2, 3, 0, 2, 0, 0],
        [2, 3, 2, 0, 0, 0],
        [2, 3, 2, 2, 3, 2]
    ]
    point = [x % (2 * np.pi) for x in point]
    best_sum = np.inf
    best_point = None
    for eq in LIST_EQUIV:
        x = np.zeros_like(point)
        for i in range(6):
            x[i] = LIST_FUN[eq[i]](point[i])
            if x[i] < 0:
                x[i] += 2 * np.pi
            elif x[i] > (2 * np.pi):
                x[i] -= 2 * np.pi
        if sum(x) == best_sum:
            # Ties are solved by smaller first components
            for i in range(6):
                if x[i] == best_point[i]:
                    continue
                elif x[i] < best_point[i]:
                    best_point = x
                    break
                else:
                    break
        elif sum(x) < best_sum:
            best_point = x
            best_sum = sum(x)
    return best_point

In [21]:
rotations_equal([0,0,0,0,0,0], [np.pi,0,0,0,0,np.pi], return_m=True)

(array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[-1.0000000e+00, -1.2246468e-16,  0.0000000e+00,  0.0000000e+00],
        [ 1.2246468e-16, -1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00, -1.0000000e+00, -1.2246468e-16],
        [ 0.0000000e+00,  0.0000000e+00,  1.2246468e-16, -1.0000000e+00]]))

---

## 3. Solving 16 equations (from a 4D rotation matrix)

In this notebook we are trying to solve a system of 16 equations using [`SymPy`](https://www.sympy.org/en/index.html) python library.  
Given a 4D dimensional matrix, we want to get 6D vector that created it.

In [19]:
import sympy as sp
from sympy import var, Matrix, Eq, solve
sp.init_printing()
import numpy as np

In [12]:
var('''Rxy, cxy, sxy,
    Rxz, cxz, sxz,
    Rxw, cxw, sxw,
    Ryz, cyz, syz,
    Ryw, cyw, syw, 
    Rzw, czw, szw''')

Rxy = Matrix([[  cxy,  -sxy, 0.0, 0.0],
              [  sxy,   cxy, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

Rxz = Matrix([[  cxz, 0.0,  -sxz, 0.0],
           [0.0, 1.0, 0.0, 0.0],
           [  sxz, 0.0,   cxz, 0.0],
           [0.0, 0.0, 0.0, 1.0]])

Rxw = Matrix([[  cxw, 0.0, 0.0,  -sxw],
           [0.0, 1.0, 0.0, 0.0],
           [0.0, 0.0, 1.0, 0.0],
           [  sxw, 0.0, 0.0,  cxw]])

Ryz = Matrix([[1.0, 0.0, 0.0, 0.0],
           [0.0,   cyz,  -syz, 0.0],
           [0.0,   syz,   cyz, 0.0],
           [0.0, 0.0, 0.0, 1.0]])

Ryw = Matrix([[1.0, 0.0, 0.0, 0.0],
           [0.0,   cyw, 0.0,  -syw],
           [0.0, 0.0, 1.0, 0.0],
           [0.0,   syw, 0.0,  cyw]])

Rzw = Matrix([[1.0, 0.0, 0.0, 0.0],
           [0.0, 1.0, 0.0, 0.0],
           [0.0, 0.0,   czw,  -szw],
           [0.0, 0.0,   szw,  czw]])

In [13]:
M = Rxy.multiply(Rxz).multiply(Rxw).multiply(Ryz).multiply(Ryw).multiply(Rzw)

In [14]:
M

⎡1.0⋅cxw⋅cxy⋅cxz  -1.0⋅cxy⋅cxz⋅sxw⋅syw + 1.0⋅cyw⋅(-1.0⋅cxy⋅sxz⋅syz - 1.0⋅cyz⋅s
⎢                                                                             
⎢1.0⋅cxw⋅cxz⋅sxy  -1.0⋅cxz⋅sxw⋅sxy⋅syw + 1.0⋅cyw⋅(1.0⋅cxy⋅cyz - 1.0⋅sxy⋅sxz⋅sy
⎢                                                                             
⎢  1.0⋅cxw⋅sxz                   1.0⋅cxz⋅cyw⋅syz - 1.0⋅sxw⋅sxz⋅syw            
⎢                                                                             
⎣    1.0⋅sxw                                1.0⋅cxw⋅syw                       

xy)  czw⋅(-1.0⋅cxy⋅cyz⋅sxz + 1.0⋅sxy⋅syz) + szw⋅(-1.0⋅cxy⋅cxz⋅cyw⋅sxw - syw⋅(-
                                                                              
z)   czw⋅(-1.0⋅cxy⋅syz - 1.0⋅cyz⋅sxy⋅sxz) + szw⋅(-1.0⋅cxz⋅cyw⋅sxw⋅sxy - syw⋅(1
                                                                              
                            1.0⋅cxz⋅cyz⋅czw + szw⋅(-1.0⋅cxz⋅syw⋅syz - 1.0⋅cyw⋅
                                                   

In [15]:
path = '../results/alignment/trajectories/'
#path = 'optML_miniproject/results/alignment/trajectories/'
file = np.load(path+'0.npy')

In [16]:
def euler6tomarix4d(a_R):
    """ Convert 6D vector containing angles to 4D rotation matrix

    a_R: tf.tensor/np.ndarray
        Vector of shape (6,)

    Returns
    -------
    R: tf.tensor/np.ndarray
        4x4 Rotation matrix corresponding to these 6 angles of rotations
    """
    xy, xz, xw, yz, yw, zw = a_R

    cxy = np.cos(xy)
    cxz = np.cos(xz)
    cxw = np.cos(xw)
    cyz = np.cos(yz)
    cyw = np.cos(yw)
    czw = np.cos(zw)

    sxy = np.sin(xy)
    sxz = np.sin(xz)
    sxw = np.sin(xw)
    syz = np.sin(yz)
    syw = np.sin(yw)
    szw = np.sin(zw)

    # Note: wasn't able to create it as simple as np.ndarrays...
    Rxy = np.array([[  cxy,  -sxy, 0.0, 0.0],
              [  sxy,   cxy, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

    Rxz = np.array([[  cxz, 0.0,  -sxz, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [  sxz, 0.0,   cxz, 0.0],
               [0.0, 0.0, 0.0, 1.0]])

    Rxw = np.array([[  cxw, 0.0, 0.0,  -sxw],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0, 1.0, 0.0],
               [  sxw, 0.0, 0.0,  cxw]])

    Ryz = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0,   cyz,  -syz, 0.0],
               [0.0,   syz,   cyz, 0.0],
               [0.0, 0.0, 0.0, 1.0]])

    Ryw = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0,   cyw, 0.0,  -syw],
               [0.0, 0.0, 1.0, 0.0],
               [0.0,   syw, 0.0,  cyw]])

    Rzw = np.array([[1.0, 0.0, 0.0, 0.0],
               [0.0, 1.0, 0.0, 0.0],
               [0.0, 0.0,   czw,  -szw],
               [0.0, 0.0,   szw,  czw]])

    R = Rxy @ Rxz @ Rxw @ Ryz @ Ryw @ Rzw

    return R

In [60]:
R = euler6tomarix4d(file[-1,:])

In [61]:
R

array([[ 0.34439141, -0.6893094 ,  0.50239565,  0.39223172],
       [ 0.72365664,  0.46871384,  0.39482803, -0.31739443],
       [ 0.55168842, -0.35996205, -0.73005563, -0.18189554],
       [-0.23098812, -0.41903128,  0.24233904, -0.84399589]])

In [52]:
R = np.eye(4).astype(np.int)

In [63]:
M

⎡1.0⋅cxw⋅cxy⋅cxz  -1.0⋅cxy⋅cxz⋅sxw⋅syw + 1.0⋅cyw⋅(-1.0⋅cxy⋅sxz⋅syz - 1.0⋅cyz⋅s
⎢                                                                             
⎢1.0⋅cxw⋅cxz⋅sxy  -1.0⋅cxz⋅sxw⋅sxy⋅syw + 1.0⋅cyw⋅(1.0⋅cxy⋅cyz - 1.0⋅sxy⋅sxz⋅sy
⎢                                                                             
⎢  1.0⋅cxw⋅sxz                   1.0⋅cxz⋅cyw⋅syz - 1.0⋅sxw⋅sxz⋅syw            
⎢                                                                             
⎣    1.0⋅sxw                                1.0⋅cxw⋅syw                       

xy)  czw⋅(-1.0⋅cxy⋅cyz⋅sxz + 1.0⋅sxy⋅syz) + szw⋅(-1.0⋅cxy⋅cxz⋅cyw⋅sxw - syw⋅(-
                                                                              
z)   czw⋅(-1.0⋅cxy⋅syz - 1.0⋅cyz⋅sxy⋅sxz) + szw⋅(-1.0⋅cxz⋅cyw⋅sxw⋅sxy - syw⋅(1
                                                                              
                            1.0⋅cxz⋅cyz⋅czw + szw⋅(-1.0⋅cxz⋅syw⋅syz - 1.0⋅cyw⋅
                                                   

In [64]:
a = Eq(M[0,0],R[0,0])
b = Eq(M[0,1],R[0,1])
c = Eq(M[0,2],R[0,2])
d = Eq(M[0,3],R[0,3])
e = Eq(M[1,0],R[1,0])
f = Eq(M[1,1],R[1,1])
g = Eq(M[1,2],R[1,2])
h = Eq(M[1,3],R[1,3])
i = Eq(M[2,0],R[2,0])
j = Eq(M[2,1],R[2,1])
k = Eq(M[2,2],R[2,2])
l = Eq(M[2,3],R[2,3])
m = Eq(M[3,0],R[3,0])
n = Eq(M[3,1],R[3,1])
o = Eq(M[3,2],R[3,2])
p = Eq(M[3,3],R[3,3])


solve([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p],(cxy, sxy,cxz, sxz,cxw, sxw,cyz, syz,cyw, syw, czw, szw), simplify=False, rational=False)

[]

#### Conclustion: 
We are unable to get the 6D vector by solving these 16 equations.